<a href="https://colab.research.google.com/github/ethanduncan65/NFL-Team-Analyzer/blob/main/NFL_Team_Analyzer_1_0b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fantasy Football FLEX Production Analyzer 1.0b - Scraping data from ESPN

Notes: 

*   This is for scraping the data and loading it into a local .csv file
*   Runtime is approx. 1 min. for one week of games



In [ ]:
from bs4 import BeautifulSoup as soup  
from urllib.request import urlopen as uReq  
import pandas as pd

In [ ]:
# store all game_ids to current date
game_ids_week1 = [401326322, 401326315, 401326308, 401326316, 401326317, 401326311, 401326318, 401326312, 401326309, 401326310, 401326314, 401326313, 401326319, 401326320, 401326323, 401326321]
game_ids_week2 = [401326127, 401326129, 401326130, 401326131, 401326353, 401326354, 401326355, 401326356, 401326128, 401326132, 401326357, 401326358, 401326359, 401326360, 401326361, 401326362]
game_ids_week3 = [401326363, 401326364, 401326365, 401326366, 401326372, 401326368, 401326369, 401326370, 401326371, 401326367, 401326373, 401326374, 401326375, 401326376, 401326377, 401326378]
game_ids_testing = [401326358]

#game_ids = game_ids_week1 + game_ids_week2 + game_ids_week3
game_ids = game_ids_testing

# go through each game and record stats & points allowed for both teams 

# create new df of average points allowed by each team

# color code df based on their distance from the average

In [ ]:
df = pd.DataFrame(columns = ['Week', 'Team', 'Opp', 'QB Pts', 'RB Pts', 'WR Pts', 'TE Pts', 'Total Pts'],  index = range(0, (len(game_ids)*2)))

Format = "Standard" #@param ["Standard", "Half-PPR", "Full-PPR"]
format_dict = {'Standard': 0,
               'Half-PPR': 0.5,
               'Full-PPR': 1}
PPR = format_dict.get(Format)
i=0
g=0

while i<(len(game_ids)*2):
  page_url = 'https://www.espn.com/nfl/boxscore/_/gameId/' + str(game_ids[g]) + ''
  # ...
  uClient = uReq(page_url)
  # ...
  page_soup = soup(uClient.read(), "html.parser")
  uClient.close()

  # tfx away team data
  away_team = page_soup.find('div', {'class': "team away"})
  away_team_name = away_team.find('span', {'class': "abbrev"})
  away_team_name_str = away_team_name.get_text()
  # tfx home team data
  home_team = page_soup.find('div', {'class': "team home"})
  home_team_name = home_team.find('span', {'class': "abbrev"})
  home_team_name_str = home_team_name.get_text()

  ### away team (column 1) scrape
  # setup counter vars for point totals
  away_qb_points = 0 
  away_rb_points = 0 
  away_wr_points = 0 
  away_te_points = 0 
  # add Team and Opp to df
  df.at[i, 'Team'] = away_team_name_str
  df.at[i, 'Opp'] = home_team_name_str
  # select the away team data/containers
  away_boxscore_div = page_soup.findAll('div', {'class': "col column-one gamepackage-away-wrap"})
  # parse the Passing container
  passing_away_data = away_boxscore_div[0].findAll('tr')
  for q in range(1,len(passing_away_data)-1):
    # scrape yards
    q_player_yds = passing_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = passing_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # scrape interceptions
    q_player_intr = passing_away_data[q].find('td', {'class': "int"}) 
    q_player_intr_str = q_player_intr.get_text()
    q_player_intr_int = int(q_player_intr_str)
    # tally points accordingly
    away_qb_points+= 0.04*q_player_yds_int + 4*q_player_td_int
    away_qb_points-= 2*q_player_intr_int
  # parse the Rushing container
  rushing_away_data = away_boxscore_div[1].findAll('tr')
  for q in range(1,len(rushing_away_data)-1):
    # scrape yards
    q_player_yds = rushing_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = rushing_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # determine player's position
    q_player_link = rushing_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
    q_player_link_str = str(q_player_link['href'])
    player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
    player_uClient = uReq(player_page_url)
    # ...
    player_page_soup = soup(player_uClient.read(), "html.parser")
    player_uClient.close()
    #
    player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
    player_position_str = player_position[1].get_text()
    # depending on player's position, tally points accordingly
    if player_position_str=='Quarterback':
      away_qb_points+= 0.1*q_player_yds_int + 6*q_player_td_int
    elif player_position_str=='Wide Receiver':
      away_wr_points+= 0.1*q_player_yds_int + 6*q_player_td_int
    else:
      away_rb_points+= 0.1*q_player_yds_int + 6*q_player_td_int 
  # parse the Receiving container
  receiving_away_data = away_boxscore_div[2].findAll('tr')
  for q in range(1,len(receiving_away_data)-1):
    # scrape receptions
    q_player_rec = receiving_away_data[q].find('td', {'class': "rec"}) 
    q_player_rec_str = q_player_rec.get_text()
    q_player_rec_int = int(q_player_rec_str)
    # scrape yards
    q_player_yds = receiving_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = receiving_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # determine player's position
    q_player_link = receiving_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
    q_player_link_str = str(q_player_link['href'])
    player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
    player_uClient = uReq(player_page_url)
    # ...
    player_page_soup = soup(player_uClient.read(), "html.parser")
    player_uClient.close()
    #
    player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
    player_position_str = player_position[1].get_text()
    # depending on player's position, tally points accordingly
    if player_position_str=='Wide Receiver':
      away_wr_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
    elif player_position_str=='Tight End':
      away_te_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
    else:
      away_rb_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
  # check for fumbles
  fumbles_away_data = away_boxscore_div[3].findAll('tr')
  # if there are fumbles, go through each one and adjust point totals accordingly 
  if len(fumbles_away_data)>2:
    for q in range(1,len(fumbles_away_data)-1):
      # scrape fumbles lost 
      q_player_fum = fumbles_away_data[q].find('td', {'class': "lost"}) 
      q_player_fum_str = q_player_fum.get_text()
      q_player_fum_int = int(q_player_fum_str)
      # if the fumble was lost (turnover) deduct points 
      if q_player_fum_int>0:
        # determine player's position
        q_player_link = fumbles_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
        q_player_link_str = str(q_player_link['href'])
        player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
        player_uClient = uReq(player_page_url)
        # ...
        player_page_soup = soup(player_uClient.read(), "html.parser")
        player_uClient.close()
        #
        player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
        player_position_str = player_position[1].get_text()
        if player_position_str=='Quarterback':
          away_qb_points-= 2*q_player_fum_int
        if player_position_str=='Running Back':
          away_rb_points-= 2*q_player_fum_int
        if player_position_str=='Wide Receiver':
          away_wr_points-= 2*q_player_fum_int
        if player_position_str=='Tight End':
          away_te_points-= 2*q_player_fum_int
  # after scraping is complete for away team, add data to df
  df.at[i, 'QB Pts'] = away_qb_points
  df.at[i, 'RB Pts'] = away_rb_points
  df.at[i, 'WR Pts'] = away_wr_points
  df.at[i, 'TE Pts'] = away_te_points
  df.at[i, 'Total Pts'] = away_qb_points + away_rb_points + away_wr_points + away_te_points
  # ...
  i+=1

  ### home team (column 2) scrape
  # setup counter vars for point totals
  away_qb_points = 0 
  away_rb_points = 0 
  away_wr_points = 0 
  away_te_points = 0 
  # add Team and Opp to df
  df.at[i, 'Team'] = home_team_name_str
  df.at[i, 'Opp'] = away_team_name_str
  # select the away team data/containers
  away_boxscore_div = page_soup.findAll('div', {'class': "col column-two gamepackage-home-wrap"})
  # parse the Passing container
  passing_away_data = away_boxscore_div[0].findAll('tr')
  for q in range(1,len(passing_away_data)-1):
    # scrape yards
    q_player_yds = passing_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = passing_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # scrape interceptions
    q_player_intr = passing_away_data[q].find('td', {'class': "int"}) 
    q_player_intr_str = q_player_intr.get_text()
    q_player_intr_int = int(q_player_intr_str)
    # tally points accordingly
    away_qb_points+= 0.04*q_player_yds_int + 4*q_player_td_int
    away_qb_points-= 2*q_player_intr_int
  # parse the Rushing container
  rushing_away_data = away_boxscore_div[1].findAll('tr')
  for q in range(1,len(rushing_away_data)-1):
    # scrape yards
    q_player_yds = rushing_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = rushing_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # determine player's position
    q_player_link = rushing_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
    q_player_link_str = str(q_player_link['href'])
    player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
    player_uClient = uReq(player_page_url)
    # ...
    player_page_soup = soup(player_uClient.read(), "html.parser")
    player_uClient.close()
    #
    player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
    player_position_str = player_position[1].get_text()
    # depending on player's position, tally points accordingly
    if player_position_str=='Quarterback':
      away_qb_points+= 0.1*q_player_yds_int + 6*q_player_td_int
    elif player_position_str=='Wide Receiver':
      away_wr_points+= 0.1*q_player_yds_int + 6*q_player_td_int
    else:
      away_rb_points+= 0.1*q_player_yds_int + 6*q_player_td_int 
  # parse the Receiving container
  receiving_away_data = away_boxscore_div[2].findAll('tr')
  for q in range(1,len(receiving_away_data)-1):
    # scrape receptions
    q_player_rec = receiving_away_data[q].find('td', {'class': "rec"}) 
    q_player_rec_str = q_player_rec.get_text()
    q_player_rec_int = int(q_player_rec_str)
    # scrape yards
    q_player_yds = receiving_away_data[q].find('td', {'class': "yds"}) 
    q_player_yds_str = q_player_yds.get_text()
    q_player_yds_int = int(q_player_yds_str)
    # scrape touchdowns
    q_player_td = receiving_away_data[q].find('td', {'class': "td"}) 
    q_player_td_str = q_player_td.get_text()
    q_player_td_int = int(q_player_td_str)
    # determine player's position
    q_player_link = receiving_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
    q_player_link_str = str(q_player_link['href'])
    player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
    player_uClient = uReq(player_page_url)
    # ...
    player_page_soup = soup(player_uClient.read(), "html.parser")
    player_uClient.close()
    #
    player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
    player_position_str = player_position[1].get_text()
    # depending on player's position, tally points accordingly
    if player_position_str=='Wide Receiver':
      away_wr_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
    elif player_position_str=='Tight End':
      away_te_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
    else:
      away_rb_points+= 0.1*q_player_yds_int + 6*q_player_td_int + PPR*q_player_rec_int
  # check for fumbles
  fumbles_away_data = away_boxscore_div[3].findAll('tr')
  # if there are fumbles, go through each one and adjust point totals accordingly 
  if len(fumbles_away_data)>2:
    for q in range(1,len(fumbles_away_data)-1):
      # scrape fumbles lost 
      q_player_fum = fumbles_away_data[q].find('td', {'class': "lost"}) 
      q_player_fum_str = q_player_fum.get_text()
      q_player_fum_int = int(q_player_fum_str)
      # if the fumble was lost (turnover) deduct points 
      if q_player_fum_int>0:
        # determine player's position
        q_player_link = fumbles_away_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
        q_player_link_str = str(q_player_link['href'])
        player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
        player_uClient = uReq(player_page_url)
        # ...
        player_page_soup = soup(player_uClient.read(), "html.parser")
        player_uClient.close()
        #
        player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
        player_position_str = player_position[1].get_text()
        if player_position_str=='Quarterback':
          away_qb_points-= 2*q_player_fum_int
        if player_position_str=='Running Back':
          away_rb_points-= 2*q_player_fum_int
        if player_position_str=='Wide Receiver':
          away_wr_points-= 2*q_player_fum_int
        if player_position_str=='Tight End':
          away_te_points-= 2*q_player_fum_int
  # after scraping is complete for away team, add data to df
  df.at[i, 'QB Pts'] = away_qb_points
  df.at[i, 'RB Pts'] = away_rb_points
  df.at[i, 'WR Pts'] = away_wr_points
  df.at[i, 'TE Pts'] = away_te_points
  df.at[i, 'Total Pts'] = away_qb_points + away_rb_points + away_wr_points + away_te_points
  # ... 
  i+=1
  g+=1

# preview df
df

In [ ]:
print(passing_away_data[1].find('td', {'class': "yds"}))
print(passing_away_data[1].find('td', {'class': "td"}))
print(passing_away_data[1].find('td', {'class': "int"}))

In [ ]:
sumX = 0.04*276 + 4*5 + 0.1*6 -2
print(sumX)

In [ ]:
df.to_csv('Season Log_standard.csv', index=False)

In [ ]:
df_seasonLog = pd.read_csv('Season Log_standard.csv')
df_seasonLog

In [ ]:
# remove duplicates to create list of all teams
teams = list(set(df_seasonLog['Opp']))
teams

# calculate averages for each team 
for t in range(0,len(teams)):
  df_t_team = df_seasonLog[df_seasonLog.Opp==teams[t]]
  t_avg_qb_pts = df_t_team['QB Pts'].mean()

# standardize the points ...

In [ ]:
df_team = df[df.Opp=='SEA']
df_team

In [ ]:
df_team['QB Pts'].mean()

In [ ]:
df_ptsAllowed = pd.DataFrame(columns = ['Week', 'Team', 'QB Pts', 'RB Pts', 'WR Pts', 'TE Pts', 'Total Pts'],  index = range(0, 32))


In [ ]:
fumbles_away_data = away_boxscore_div[3].findAll('tr')
len(fumbles_away_data)

In [ ]:
wr_data = away_boxscore_div[2].findAll('tr')
#wr_data[2]

# receiving scrape
for q in range(1,len(wr_data)-1):
  # scrape receptions
  q_player_rec = wr_data[q].find('td', {'class': "rec"}) 
  q_player_rec_str = q_player_rec.get_text()
  q_player_rec_int = int(q_player_rec_str)
  # scrape yards
  q_player_yds = wr_data[q].find('td', {'class': "yds"}) 
  q_player_yds_str = q_player_yds.get_text()
  q_player_yds_int = int(q_player_yds_str)
  # scrape touchdowns
  q_player_td = wr_data[q].find('td', {'class': "td"}) 
  q_player_td_str = q_player_td.get_text()
  q_player_td_int = int(q_player_td_str)
  # determine player's position
  q_player_link = wr_data[q].find('a', {'name': "&lpos=nfl:game:boxscore:playercard"})
  q_player_link_str = str(q_player_link['href'])
  player_page_url = q_player_link_str.split('_')[0] + 'bio/_' + q_player_link_str.split('_')[1]
  player_uClient = uReq(player_page_url)
  # ...
  player_page_soup = soup(player_uClient.read(), "html.parser")
  player_uClient.close()
  #
  player_position = player_page_soup.findAll('span', {'class': "dib flex-uniform mr3 clr-gray-01"})
  player_position_str = player_position[1].get_text()
  print(player_position_str)

  #print(q_player_link['href'])

In [ ]:
q_player_rec = wr_data[2].find('td', {'class': "yds"}) 
q_player_rec_str = q_player_rec.get_text()

q_player_rec_str

In [ ]:
player_page_url = 'https://www.espn.com/nfl/player/gamelog/_/id/4047650/dk-metcalf'
# ...
player_uClient = uReq(player_page_url)
# ...
player_page_soup = soup(player_uClient.read(), "html.parser")
player_uClient.close()

player_stats = player_page_soup.findAll('td', {'class': "Table__TD"})
player_position = player_page_soup.findAll('th', {'class': "tc Table__TH"})
player_position_str = player_position[0].get_text()

In [ ]:
df_player = pd.DataFrame(columns = ['Week', 'Team', 'Opp', 'Tgt', 'Rec', 'Pass Yds', 'Pass TD', 'Car', 'Rush Yds', 'Rush TD', 'Points', '% Share'],  index = range(0, numWeeks))

player_page_url = 'https://www.espn.com/nfl/player/gamelog/_/id/4047650/dk-metcalf'
# ...
player_uClient = uReq(player_page_url)
# ...
player_page_soup = soup(player_uClient.read(), "html.parser")
player_uClient.close()

#player_stats = player_page_soup.find('tr', {'class': "Table__TR Table__TR--sm Table__even"})
#player_statsX = player_page_soup.findAll('td', {'class': "Table__TD"})
#player_statsX[0].get_text()
#player_statsX[4].get_text()
#player_statsX[36].get_text()



# determine if player is RB or WR/TE
player_stats = player_page_soup.findAll('td', {'class': "Table__TD"})
player_position = player_page_soup.findAll('th', {'class': "tc Table__TH"})
player_position_str = player_position[0].get_text()
if player_position_str=='Rushing':
  #
  k=0
  for r in range(0, numWeeks):
    # add week, team, opp
    df_player.at[r, 'Tgt'] = player_stats[9+k].get_text()
    df_player.at[r, 'Rec'] = player_stats[8+k].get_text()
    df_player.at[r, 'Pass Yds'] = player_stats[10+k].get_text()
    df_player.at[r, 'Pass TD'] = player_stats[9+k].get_text()
    df_player.at[r, 'Car'] = player_stats[3+k].get_text()
    df_player.at[r, 'Rush Yds'] = player_stats[4+k].get_text()
    df_player.at[r, 'Rush TD'] = player_stats[6+k].get_text()
    k+=18
else:
  #
  k=0
  for r in range(0, numWeeks):
    # add week, team, opp
    df_player.at[r, 'Tgt'] = player_stats[4+k].get_text()
    df_player.at[r, 'Rec'] = player_stats[3+k].get_text()
    df_player.at[r, 'Pass Yds'] = player_stats[5+k].get_text()
    df_player.at[r, 'Pass TD'] = player_stats[7+k].get_text()
    df_player.at[r, 'Car'] = player_stats[9+k].get_text()
    df_player.at[r, 'Rush Yds'] = player_stats[10+k].get_text()
    df_player.at[r, 'Rush TD'] = player_stats[13+k].get_text()
    k+=18

# reverse order of df_player to align with format of df's below

In [ ]:
Team = "Seattle Seahawks" #@param ["Seattle Seahawks", "San Francisco 49ers"]
vsTeam = "San Francisco 49ers" #@param ["Seattle Seahawks", "San Francisco 49ers"]
Format = "Standard" #@param ["Standard", "Half-PPR", "Full-PPR"]
team_sched_dict = {"Seattle Seahawks": [401326318, 401326360, 401326376],
                   "San Francisco 49ers": [401326317, 401326355, 401326377]}
game_ids = team_sched_dict.get(Team)
vs_game_ids = team_sched_dict.get(vsTeam)

team_name_dict = {"Seattle Seahawks": 'SEA',
                  "San Francisco 49ers": 'SF'}
team_abbrev = team_name_dict.get(Team)
vs_team_abbrev = team_name_dict.get(vsTeam)